# Test of the High-Level API

For development purposes

In [50]:
%load_ext autoreload
%autoreload 2
from __future__ import annotations

import pandas as pd

from tab_err.api import high_level

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
def show_result(original_df: pd.DataFrame, perturbed_df: pd.DataFrame, error_mask: pd.DataFrame | None = None) -> pd.DataFrame:
    """Simple helper function to show DataFrames after perturbing them."""
    return (
        pd.concat([original_df, perturbed_df], keys=["original", "perturbed"], axis=1)
        if error_mask is None
        else pd.concat([original_df, perturbed_df, error_mask], keys=["original", "perturbed", "error_mask"], axis=1)
    )

In [52]:
df_typist_book_title = pd.DataFrame(
    {
        "typist": ["Alice", "Alice", "Alice", "Bob", "Bob", "Bob"],
        "book_title": ["To Kill a Mockingbird", "1984", "Pride and Prejudice", "The Great Gatsby", "Moby-Dick", "The Catcher in the Rye"],
        "rating": [1.0, 3.0, 3.0, 4.0, 2.0, 1.0],
    }
)



print(df_typist_book_title.dtypes)

typist         object
book_title     object
rating        float64
dtype: object


### TEST

In [53]:
df_corrupted, error_mask = high_level.create_errors(df_typist_book_title, 0.5)
show_result(df_typist_book_title, df_corrupted, error_mask)

Column-type dict:  {'typist': [<tab_err.error_type._extraneous.Extraneous object at 0x7f60616cb010>, <tab_err.error_type._mojibake.Mojibake object at 0x7f60616ca020>, <tab_err.error_type._replace.Replace object at 0x7f60616c8400>, <tab_err.error_type._typo.Typo object at 0x7f60616cb790>, <tab_err.error_type._missing.MissingValue object at 0x7f60616caad0>], 'book_title': [<tab_err.error_type._extraneous.Extraneous object at 0x7f60616cb010>, <tab_err.error_type._mojibake.Mojibake object at 0x7f60616ca020>, <tab_err.error_type._replace.Replace object at 0x7f60616c8400>, <tab_err.error_type._typo.Typo object at 0x7f60616cb790>, <tab_err.error_type._missing.MissingValue object at 0x7f60616caad0>], 'rating': [<tab_err.error_type._add_delta.AddDelta object at 0x7f60616c9ea0>, <tab_err.error_type._outlier.Outlier object at 0x7f60616c9390>, <tab_err.error_type._wrong_unit.WrongUnit object at 0x7f60616cae60>, <tab_err.error_type._missing.MissingValue object at 0x7f60616caad0>]}
Column-mech dict:

/mnt/c/Users/njcha/OneDrive/BHT/Research/tab_err/tab_err/api/high_level.py:130: UserWarning: With an error rate for the column (typist) of: 0.025 and a length of: 6, 0 errors will be introduced.
  col_error_rates = build_column_error_rate_dictionary(data, max_error_rate, col_num_models)
/mnt/c/Users/njcha/OneDrive/BHT/Research/tab_err/tab_err/api/high_level.py:130: UserWarning: With an error rate for the column (book_title) of: 0.025 and a length of: 6, 0 errors will be introduced.
  col_error_rates = build_column_error_rate_dictionary(data, max_error_rate, col_num_models)
/mnt/c/Users/njcha/OneDrive/BHT/Research/tab_err/tab_err/api/high_level.py:130: UserWarning: With an error rate for the column (rating) of: 0.03125 and a length of: 6, 0 errors will be introduced.
  col_error_rates = build_column_error_rate_dictionary(data, max_error_rate, col_num_models)
/mnt/c/Users/njcha/OneDrive/BHT/Research/tab_err/tab_err/error_type/_error_type.py:67: UserWarning: self.config.extraneous_value_t

original                                perturbed                          \
    typist              book_title rating    typist              book_title   
0    Alice   To Kill a Mockingbird    1.0     Alice   To Kill a Mockingbird   
1    Alice                    1984    3.0     Alice                    1984   
2    Alice     Pride and Prejudice    3.0     Alice     Pride and Prejudice   
3      Bob        The Great Gatsby    4.0       Bob        The Great Gatsby   
4      Bob               Moby-Dick    2.0       Bob               Moby-Dick   
5      Bob  The Catcher in the Rye    1.0       Bob  The Catcher in the Rye   

         error_mask                    
  rating     typist book_title rating  
0    1.0      False      False  False  
1    3.0      False      False  False  
2    3.0      False      False  False  
3    4.0      False      False  False  
4    2.0      False      False  False  
5    1.0      False      False  False

# Testing of High Level API with a large dataset

In [54]:
# Read in data
df_clean = pd.read_csv("./clean_beers.csv")
df_clean.head()

,index,id,beer_name,style,ounces,abv,ibu,brewery_id,brewery_name,city,state
0,1,1436,Pub Beer,American Pale Lager,12.0,0.050,NaN,408,10 Barrel Brewing Company,Bend,OR
1,2,2265,Devil's Cup,American Pale Ale (APA),12.0,0.066,NaN,177,18th Street Brewery,Gary,IN
2,3,2264,Rise of the Phoenix,American IPA,12.0,0.071,NaN,177,18th Street Brewery,Gary,IN
3,4,2263,Sinister,American Double / Imperial IPA,12.0,0.090,NaN,177,18th Street Brewery,Gary,IN
4,5,2262,Sex and Candy,American IPA,12.0,0.075,NaN,177,18th Street Brewery,Gary,IN


In [55]:
df_clean = df_clean.drop(columns = ["index", "id", "ounces", "ibu", "brewery_id", "city"])
df_clean.head()

,beer_name,style,abv,brewery_name,state
0,Pub Beer,American Pale Lager,0.050,10 Barrel Brewing Company,OR
1,Devil's Cup,American Pale Ale (APA),0.066,18th Street Brewery,IN
2,Rise of the Phoenix,American IPA,0.071,18th Street Brewery,IN
3,Sinister,American Double / Imperial IPA,0.090,18th Street Brewery,IN
4,Sex and Candy,American IPA,0.075,18th Street Brewery,IN


In [56]:
# Add a date time column, just all the same date
df_clean["arbitrary_date"] = pd.to_datetime("2025-02-20 14:30:00")
df_clean.head()

,beer_name,style,abv,brewery_name,state,arbitrary_date
0,Pub Beer,American Pale Lager,0.050,10 Barrel Brewing Company,OR,2025-02-20 14:30:00
1,Devil's Cup,American Pale Ale (APA),0.066,18th Street Brewery,IN,2025-02-20 14:30:00
2,Rise of the Phoenix,American IPA,0.071,18th Street Brewery,IN,2025-02-20 14:30:00
3,Sinister,American Double / Imperial IPA,0.090,18th Street Brewery,IN,2025-02-20 14:30:00
4,Sex and Candy,American IPA,0.075,18th Street Brewery,IN,2025-02-20 14:30:00


In [57]:
df_clean["beer_name"] = df_clean["beer_name"].astype(str)
df_clean["style"] = df_clean["style"].astype(str)
df_clean["brewery_name"] = df_clean["brewery_name"].astype(str)
df_clean["state"] = df_clean["state"].astype(str)
df_clean.dtypes

beer_name                 object
style                     object
abv                      float64
brewery_name              object
state                     object
arbitrary_date    datetime64[ns]
dtype: object

In [58]:
df_dirty, error_mask = high_level.create_errors(df_clean, max_error_rate=0.75)
show_result(df_clean, df_dirty, error_mask)

Column-type dict:  {'beer_name': [<tab_err.error_type._extraneous.Extraneous object at 0x7f60616c94e0>, <tab_err.error_type._mojibake.Mojibake object at 0x7f60616cb550>, <tab_err.error_type._replace.Replace object at 0x7f60616c8ca0>, <tab_err.error_type._typo.Typo object at 0x7f60616cba30>, <tab_err.error_type._missing.MissingValue object at 0x7f60616cbc70>], 'style': [<tab_err.error_type._extraneous.Extraneous object at 0x7f60616c94e0>, <tab_err.error_type._mojibake.Mojibake object at 0x7f60616cb550>, <tab_err.error_type._replace.Replace object at 0x7f60616c8ca0>, <tab_err.error_type._typo.Typo object at 0x7f60616cba30>, <tab_err.error_type._missing.MissingValue object at 0x7f60616cbc70>], 'abv': [<tab_err.error_type._add_delta.AddDelta object at 0x7f60616c9b40>, <tab_err.error_type._outlier.Outlier object at 0x7f60616ca2c0>, <tab_err.error_type._wrong_unit.WrongUnit object at 0x7f60616cbcd0>, <tab_err.error_type._missing.MissingValue object at 0x7f60616cbc70>], 'brewery_name': [<tab_

/mnt/c/Users/njcha/OneDrive/BHT/Research/tab_err/tab_err/error_type/_error_type.py:67: UserWarning: self.config.extraneous_value_template is not set. Choosing a random string to augment.
  return self._apply(data, error_mask, column)
/mnt/c/Users/njcha/OneDrive/BHT/Research/tab_err/tab_err/error_type/_error_type.py:67: UserWarning: self.config.add_delta_value is none, sampling a random delta value uniformly from the range of column: abv
  return self._apply(data, error_mask, column)
/mnt/c/Users/njcha/OneDrive/BHT/Research/tab_err/tab_err/error_type/_error_type.py:67: UserWarning: No scaling function was supplied for WrongUnit, defaulting to multiplication by 10.0
  return self._apply(data, error_mask, column)


original                                         \
                 beer_name                           style    abv   
0                 Pub Beer             American Pale Lager  0.050   
1              Devil's Cup         American Pale Ale (APA)  0.066   
2      Rise of the Phoenix                    American IPA  0.071   
3                 Sinister  American Double / Imperial IPA  0.090   
4            Sex and Candy                    American IPA  0.075   
...                    ...                             ...    ...   
2405             Belgorado                     Belgian IPA  0.067   
2406         Rail Yard Ale        American Amber / Red Ale  0.052   
2407       B3K Black Lager                     Schwarzbier  0.055   
2408   Silverback Pale Ale         American Pale Ale (APA)  0.055   
2409  Rail Yard Ale (2009)        American Amber / Red Ale  0.052   

                                                           \
                   brewery_name state      arbitrary_date   
0     10 Barrel Brewing Company    OR 2025-02-20 14:30:00   
1           18th Street Brewery    IN 2025-02-20 14:30:00   
2           18th Street Brewery    IN 2025-02-20 14:30:00   
3           18th Street Brewery    IN 2025-02-20 14:30:00   
4           18th Street Brewery    IN 2025-02-20 14:30:00   
...                         ...   ...                 ...   
2405    Wynkoop Brewing Company    CO 2025-02-20 14:30:00   
2406    Wynkoop Brewing Company    CO 2025-02-20 14:30:00   
2407    Wynkoop Brewing Company    CO 2025-02-20 14:30:00   
2408    Wynkoop Brewing Company    CO 2025-02-20 14:30:00   
2409    Wynkoop Brewing Company    CO 2025-02-20 14:30:00   

                   perturbed                                            \
                   beer_name                           style       abv   
0                   Pub Beer             American Pale Lager  0.050000   
1                Devil's Cup                            <NA>  1.559647   
2     \Rise of the Phoenix$)                    Amerixan IPA  0.071000   
3                   Sinister  American Double / Imperial IPA  0.900000   
4                       <NA>                    American IPA  0.075000   
...                      ...                             ...       ...   
2405               Belgorado                  \Belgian IPA$)  0.670000   
2406           Rail Yard Ale        American Amber / Red Ale  1.545647   
2407         B3K Black Lager                     Schwarzbier  1.548647   
2408     Silverback Pale Ale                            <NA>  0.055000   
2409    Rail Yard Ale (2009)                            <NA>  1.545647   

                                                                      \
                   brewery_name  state                arbitrary_date   
0     10 Barrel Brewing Companz     OR 2025-02-20 14:30:00.000000000   
1        \18th Street Brewery$)     IN 2037-07-15 16:27:49.696892416   
2           18th Street Brewery  \IN$) 2025-02-20 14:30:00.000000000   
3           18th Street Brewery     IN 2025-02-20 14:30:00.000000000   
4           18th Street Brewerz     IN 2025-02-20 14:30:00.000000000   
...                         ...    ...                           ...   
2405                       <NA>   <NA> 2025-02-20 14:30:00.000000000   
2406    Wynkoop Brewing Compaby     CO                           NaT   
2407                       <NA>     CO                           NaT   
2408    Wynkoop Brewing Company     CP 2061-01-07 20:47:59.946644480   
2409    Wynkoop Brewing Fompany   <NA> 2025-02-20 14:30:00.000000000   

     error_mask                                                   
      beer_name  style    abv brewery_name  state arbitrary_date  
0         False   True  False         True  False           True  
1          True   True   True         True  False           True  
2          True   True  False        False   True          False  
3         False  False   True         True  False          False  
4          True  False  

In [59]:
error_mask.mean(axis=None)
error_mask.mean(axis=0)

beer_name         0.740664
style             0.740664
abv               0.743568
brewery_name      0.740664
state             0.740664
arbitrary_date    0.749378
dtype: float64